# Data

In [1]:
# https://www.analyticsvidhya.com/blog/2021/06/image-recognition-using-pytorch-lightning/
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
import torchvision
import torch
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
from random import randint
import albumentations as A
from albumentations.pytorch import ToTensorV2

C:\Users\debora.assis\Anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Não foi possível encontrar o procedimento especificado
  warn(f"Failed to load image Python extension: {e}")


# Load dataset

In [2]:
# Data train 
path_image = 'C:/Users/debora.assis/Documents/UNet-SegRD/diaretdb1_v_1_1/ddb1_fundusimages'
path_mask = 'C:/Users/debora.assis/Documents/UNet-SegRD/diaretdb1_v_1_1/ddb1_groundtruth/hardexudates'
img_list_train = os.listdir(os.path.join(path_image))

In [3]:
transformations = {
    'test': transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize((224,224)),
#                         transforms.Grayscale(1),
                        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        transforms.ToTensor()]),
     'train': transforms.Compose([
                           transforms.ToPILImage(),
                           transforms.Resize((224,224)),
                           transforms.RandomRotation(degrees = 45),
                           transforms.RandomHorizontalFlip(p = 0.005),
                           transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#                            transforms.Grayscale(1),
                           transforms.ToTensor()
                           ])
}


In [4]:

class RD_SegmentationDataset(Dataset):
    def __init__(self, img_list, image_dir, mask_dir, transform=None, target_transform=None):
        
        self.data_list = img_list
        self.data_img = image_dir
        self.data_mask = mask_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data_img)

    def __getitem__(self, idx):

        path_image = os.path.join(self.data_img, self.data_list[idx])
#         print(path_image)
        image = cv2.imread(path_image)        
        image = cv2.resize(image, (224,224))
        image = torch.as_tensor(image, dtype=torch.uint8)
        image = image.permute(2,0,1)

        path_mask = os.path.join(self.data_mask, self.data_list[idx])
#         print(path_mask)
        mask = cv2.imread(path_mask,0)
        mask = cv2.resize(mask, (224,224))
        mask = cv2.threshold(mask, 240, 255, cv2.THRESH_BINARY)[1]
        mask = mask/255
        
        mask = np.expand_dims(mask, 2)
        mask = torch.as_tensor(mask, dtype=torch.uint8)
        mask = mask.permute(2,0,1)

        return {'image':image, 'mask':mask}
    
RD_dataset = RD_SegmentationDataset(img_list_train, path_image, path_mask, transformations['train'])
RD_dataloader = DataLoader(RD_dataset, batch_size=4, shuffle=True)

In [5]:
import numpy as np
# Get a batch of training data
batch = next(iter(RD_dataloader))
inputs = batch['image']
masks = batch['mask']

In [6]:
np.shape(inputs)

torch.Size([4, 3, 224, 224])

In [7]:
import pytorch_lightning as pl
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class

    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer

    def prepare_batch(self, batch):
        inputs = batch["image"].float()
        masks = batch["mask"].float()
        return inputs, masks
#         return batch["image"] , batch["mask"]

    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

In [8]:
import monai

unet = monai.networks.nets.UNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    channels=(8, 16, 32,64),
    strides=(2, 2, 2),
)

model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor="train_loss",
)
trainer = pl.Trainer(
    gpus=0,
#     precision='bf16',
    callbacks=[early_stopping]
)
trainer.logger._default_hp_metric = False

C:\Users\debora.assis\Anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
from datetime import datetime

train_loader = DataLoader(RD_dataset, batch_size=4, shuffle=True)

start = datetime.now()
print("Training started at", start)
trainer.fit(model=model, train_dataloaders=train_loader)

print("Training duration:", datetime.now() - start)

C:\Users\debora.assis\Anaconda3\lib\site-packages\pytorch_lightning\loops\utilities.py:89: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
C:\Users\debora.assis\Anaconda3\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:107: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 40.8 K
1 | criterion | DiceCELoss | 0     
-----------------------------------------
40.8 K    Trainable params
0         Non-trainable params
40.8 K    Total params
0.163     Total estimated model params size (MB)


Training started at 2023-04-05 19:34:10.596196


C:\Users\debora.assis\Anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\debora.assis\Anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:1892: PossibleUserWarning: The number of training batches (19) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

C:\Users\debora.assis\Anaconda3\lib\site-packages\monai\losses\dice.py:138: UserWarning: single channel prediction, `softmax=True` ignored.
  warnings.warn("single channel prediction, `softmax=True` ignored.")


Training duration: 0:00:29.630093
